# Implémentation des Générateurs de Nombres Aléatoires et Analyse de la Qualité

Ce notebook présente :
1. L'implémentation de 7 générateurs (PRNG, CSPRNG, hybride)
2. Une suite de 4 tests statistiques pour évaluer leur qualité
3. Des visualisations comparatives
4. 4 attaques pédagogiques démontrant les faiblesses des PRNG non sécurisés

**Note de responsabilité** : Les attaques présentées ici sont réalisées dans un cadre strictement pédagogique et sont interdites sur des systèmes réels.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# Ajouter le répertoire racine au path
sys.path.insert(0, os.path.abspath('.'))

# Import des générateurs (fonctions)
from generators.lcg import lcg_generate_bytes
from generators.mersenne_twister import mt_generate_bytes
from generators.nist_drbg import drbg_instantiate, drbg_generate
from generators.bbs import bbs_generate_bytes
from generators.os_random import os_generate_bytes
from generators.xor_nrbg import xor_nrbg_generate_bytes
from generators.box_muller import box_muller_generate

# Import des tests statistiques
from tests_statistiques import shannon_entropy, chi_squared_test, autocorrelation, ks_test, run_all_tests

# Style des graphiques
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-v0_8-darkgrid')

print("Imports réussis !")

---
## 1. Instanciation des générateurs

On instancie chaque générateur et on génère 100 000 octets pour les tests.

In [ ]:
N_BYTES = 100_000
SEED = 42

# Génération de 100 000 octets par générateur (appels de fonctions)
print("Génération des données pour chaque générateur...")

data = {}

print("  LCG...", end=" ")
data["LCG"], _ = lcg_generate_bytes(N_BYTES, seed=SEED)
print("OK")

print("  Mersenne Twister...", end=" ")
data["Mersenne Twister"], _ = mt_generate_bytes(N_BYTES, seed=SEED)
print("OK")

print("  Hash_DRBG (SHA-256)...", end=" ")
drbg_state = drbg_instantiate(entropy=b'A'*55, nonce=b'B'*28)
data["Hash_DRBG (SHA-256)"], _ = drbg_generate(drbg_state, N_BYTES)
print("OK")

print("  Blum-Blum-Shub...", end=" ")
data["Blum-Blum-Shub"] = bbs_generate_bytes(N_BYTES, seed=12345)
print("OK")

print("  os.urandom...", end=" ")
data["os.urandom"] = os_generate_bytes(N_BYTES)
print("OK")

print("  XOR NRBG...", end=" ")
data["XOR NRBG"] = xor_nrbg_generate_bytes(N_BYTES, seed=SEED)
print("OK")

print(f"\nTous les générateurs ont produit {N_BYTES} octets.")

---
## 2. Tests statistiques

On exécute les 4 tests sur chaque générateur : entropie de Shannon, chi-carré, autocorrélation et Kolmogorov-Smirnov.

In [ ]:
# Exécuter tous les tests pour chaque générateur
all_results = {}
for name, d in data.items():
    all_results[name] = run_all_tests(d, name)

### 2.1 Tableau comparatif

In [ ]:
# Tableau comparatif
print(f"{'Générateur':<25} {'Entropie':>10} {'Chi² p-val':>12} {'AutoCorr':>10} {'KS p-val':>10} {'Score':>7}")
print("─" * 80)

for name, res in all_results.items():
    entropy = res['shannon']['entropy']
    chi2_p = res['chi2']['p_value']
    autocorr_max = res['autocorrelation']['max_abs']
    ks_p = res['ks']['p_value']
    
    # Score : nombre de tests passés sur 4
    score = sum([
        entropy > 7.9,
        res['chi2']['passed'],
        res['autocorrelation']['passed'],
        res['ks']['passed'],
    ])
    
    print(f"{name:<25} {entropy:>10.4f} {chi2_p:>12.4f} {autocorr_max:>10.4f} {ks_p:>10.4f} {score:>5}/4")

---
## 3. Visualisations

### 3.1 Histogrammes de distribution des octets

Un bon générateur doit produire une distribution uniforme sur [0, 255].

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (name, d) in enumerate(data.items()):
    ax = axes[idx]
    ax.hist(list(d), bins=256, range=(0, 255), density=True, alpha=0.7, color=f'C{idx}')
    ax.axhline(y=1/256, color='red', linestyle='--', linewidth=1, label='Uniforme théorique')
    ax.set_title(name, fontsize=12, fontweight='bold')
    ax.set_xlabel('Valeur octet')
    ax.set_ylabel('Densité')
    ax.legend(fontsize=8)

plt.suptitle('Distribution des octets par générateur', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('histogrammes_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

### 3.2 Scatter plots (x_n vs x_{n+1})

Un bon générateur ne doit montrer aucune structure dans le graphe (x_n, x_{n+1}). Le LCG produit des motifs linéaires caractéristiques.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

N_SCATTER = 5000  # Nombre de points pour le scatter plot

for idx, (name, d) in enumerate(data.items()):
    ax = axes[idx]
    values = list(d[:N_SCATTER + 1])
    x = values[:-1]
    y = values[1:]
    ax.scatter(x, y, s=0.5, alpha=0.5, color=f'C{idx}')
    ax.set_title(name, fontsize=12, fontweight='bold')
    ax.set_xlabel('$x_n$')
    ax.set_ylabel('$x_{n+1}$')
    ax.set_xlim(0, 255)
    ax.set_ylim(0, 255)

plt.suptitle('Scatter plot $x_n$ vs $x_{n+1}$ (corrélations séquentielles)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('scatter_plots.png', dpi=150, bbox_inches='tight')
plt.show()

### 3.3 Graphiques d'autocorrélation

In [ ]:
lags = list(range(1, 33))

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, (name, d) in enumerate(data.items()):
    ax = axes[idx]
    result = autocorrelation(d, lags=lags)
    corr_values = [result['correlations'][lag] for lag in lags]
    
    ax.bar(lags, corr_values, color=f'C{idx}', alpha=0.7)
    # Seuil de significativité (approximation)
    threshold = 2 / np.sqrt(len(d))
    ax.axhline(y=threshold, color='red', linestyle='--', linewidth=1, label=f'±{threshold:.4f}')
    ax.axhline(y=-threshold, color='red', linestyle='--', linewidth=1)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.set_title(name, fontsize=12, fontweight='bold')
    ax.set_xlabel('Lag')
    ax.set_ylabel('Autocorrélation')
    ax.legend(fontsize=8)

plt.suptitle('Autocorrélation par lag', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('autocorrelation.png', dpi=150, bbox_inches='tight')
plt.show()

### 3.4 Distribution gaussienne Box-Muller

Vérification que la transformée de Box-Muller produit bien une distribution N(0,1).

In [ ]:
from scipy.stats import norm

# Générer 50 000 valeurs gaussiennes via Box-Muller
gaussian_values = box_muller_generate(50_000, seed=42)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme + courbe théorique
ax1.hist(gaussian_values, bins=100, density=True, alpha=0.7, color='steelblue', label='Box-Muller')
x = np.linspace(-4, 4, 1000)
ax1.plot(x, norm.pdf(x), 'r-', linewidth=2, label='N(0,1) théorique')
ax1.set_title('Distribution Box-Muller vs N(0,1)', fontweight='bold')
ax1.set_xlabel('Valeur')
ax1.set_ylabel('Densité')
ax1.legend()

# QQ-plot
from scipy.stats import probplot
probplot(gaussian_values, dist="norm", plot=ax2)
ax2.set_title('QQ-Plot (Box-Muller vs Normal)', fontweight='bold')

plt.tight_layout()
plt.savefig('box_muller_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Statistiques Box-Muller (50 000 valeurs) :")
print(f"  Moyenne  : {np.mean(gaussian_values):.4f} (attendu: 0.0)")
print(f"  Écart-type: {np.std(gaussian_values):.4f} (attendu: 1.0)")
print(f"  Min      : {np.min(gaussian_values):.4f}")
print(f"  Max      : {np.max(gaussian_values):.4f}")

### 3.5 Comparaison visuelle de l'entropie

In [ ]:
names = list(all_results.keys())
entropies = [all_results[n]['shannon']['entropy'] for n in names]
chi2_passed = [all_results[n]['chi2']['passed'] for n in names]
colors = ['green' if p else 'red' for p in chi2_passed]

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.barh(names, entropies, color=colors, alpha=0.8)
ax.axvline(x=8.0, color='blue', linestyle='--', linewidth=1, label='Maximum théorique (8.0)')
ax.axvline(x=7.9, color='orange', linestyle='--', linewidth=1, label='Seuil acceptable (7.9)')
ax.set_xlabel('Entropie (bits/octet)')
ax.set_title('Entropie de Shannon par générateur', fontweight='bold')
ax.legend()

# Ajouter les valeurs sur les barres
for bar, val in zip(bars, entropies):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
            f'{val:.4f}', va='center', fontsize=10)

ax.set_xlim(0, 8.5)
plt.tight_layout()
plt.savefig('entropie_comparaison.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Attaques pédagogiques

**AVERTISSEMENT** : Ces attaques sont réalisées dans un cadre strictement pédagogique.

### 4.1 Récupération de la graine LCG

**Modèle de menace** : L'attaquant connaît les paramètres du LCG et un fragment de texte clair. Il retrouve la graine par brute-force et déchiffre tout le message.

In [ ]:
from attacks.lcg_seed_recovery import demo_lcg_seed_recovery

lcg_results = demo_lcg_seed_recovery()

### 4.2 Reconstruction d'état MT19937

**Modèle de menace** : L'attaquant observe 624 sorties consécutives du Mersenne Twister et reconstruit l'état interne complet pour prédire toutes les sorties futures.

In [ ]:
from attacks.mt_state_recovery import demo_mt_state_recovery

mt_results = demo_mt_state_recovery()

### 4.3 Réutilisation de nonce en AES-CTR

**Modèle de menace** : Deux messages sont chiffrés avec la même clé et le même nonce en mode AES-CTR. L'attaquant récupère le XOR des clairs et utilise le crib-dragging.

In [ ]:
from attacks.aes_ctr_nonce_reuse import demo_aes_ctr_nonce_reuse

ctr_results = demo_aes_ctr_nonce_reuse()

### 4.4 IV prévisible en AES-CBC

**Modèle de menace** : Le système utilise un IV prévisible (compteur). L'attaquant peut soumettre des messages choisis et détecter si un message cible a été chiffré précédemment.

In [ ]:
from attacks.aes_cbc_iv_attack import demo_aes_cbc_iv_attack

cbc_results = demo_aes_cbc_iv_attack()

---
## 5. Tableau comparatif final et recommandations

In [ ]:
print("=" * 90)
print(" TABLEAU COMPARATIF FINAL ".center(90, "="))
print("=" * 90)
print()
print(f"{'Générateur':<25} {'Type':<12} {'Entropie':>8} {'χ²':>6} {'AC':>6} {'KS':>6} {'Crypto':>8}")
print("─" * 90)

gen_types = {
    "LCG": ("PRNG", "Non"),
    "Mersenne Twister": ("PRNG", "Non"),
    "Hash_DRBG (SHA-256)": ("CSPRNG", "Oui"),
    "Blum-Blum-Shub": ("CSPRNG", "Oui"),
    "os.urandom": ("TRNG/OS", "Oui"),
    "XOR NRBG": ("Hybride", "Oui*"),
}

for name, res in all_results.items():
    gtype, crypto = gen_types[name]
    entropy = res['shannon']['entropy']
    chi2 = "PASS" if res['chi2']['passed'] else "FAIL"
    ac = "PASS" if res['autocorrelation']['passed'] else "FAIL"
    ks = "PASS" if res['ks']['passed'] else "FAIL"
    print(f"{name:<25} {gtype:<12} {entropy:>8.4f} {chi2:>6} {ac:>6} {ks:>6} {crypto:>8}")

print()
print("* XOR NRBG : sécurité dépend de la meilleure source incluse")
print()
print("=" * 90)
print(" RECOMMANDATIONS ".center(90, "="))
print("=" * 90)
print("""
1. Pour la cryptographie : utiliser os.urandom, secrets (Python), ou un CSPRNG
   validé (Hash_DRBG, CTR_DRBG).

2. NE JAMAIS utiliser LCG ou Mersenne Twister pour la génération de clés,
   nonces, IV ou tout usage de sécurité.

3. Toujours utiliser des nonces/IV uniques et imprévisibles en AES-CTR/CBC.

4. La construction XOR NRBG améliore la robustesse si au moins une source
   est fiable, mais ne remplace pas un vrai CSPRNG.

5. Blum-Blum-Shub offre une sécurité théorique forte mais est trop lent
   pour un usage pratique (préférer Hash_DRBG ou os.urandom).
""")